In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openai

In [4]:
nyt = pd.read_csv("nyt_full.tsv", sep = '\t')
nyt.head()

,year,week,rank,title_id,title,author
0,1931,1931-10-12,1,6477,THE TEN COMMANDMENTS,Warwick Deeping
1,1931,1931-10-12,2,1808,FINCHE'S FORTUNE,Mazo de la Roche
2,1931,1931-10-12,3,5304,THE GOOD EARTH,Pearl S. Buck
3,1931,1931-10-12,4,4038,SHADOWS ON THE ROCK,Willa Cather
4,1931,1931-10-12,5,3946,SCARMOUCHE THE KING MAKER,Rafael Sabatini


In [6]:
nyt.author.nunique()

2210

In [8]:
print(nyt.author)

0                        Warwick Deeping
1                       Mazo de la Roche
2                          Pearl S. Buck
3                           Willa Cather
4                        Rafael Sabatini
                      ...               
60381                             Halsey
60382                       Brit Bennett
60383                        Delia Owens
60384                    Fredrik Backman
60385    Clive Cussler and Boyd Morrison
Name: author, Length: 60386, dtype: object


In [50]:
# replace written & illustrated by 
nyt.author = nyt.author.replace("written and illustrated by ","", regex = True)

## separate complicated entries

In [61]:
#if there is a preposition, there's complication
nyt['problem'] = nyt.author.str.contains(' and | with | by ', regex = True, case = False)
print("how many problem entries", nyt.problem.sum())
print(nyt.problem.sum()/len(nyt))

# new dataframe with just problems
nyt_review = nyt[nyt.problem == True]
print("how many unique problems", len(nyt_review.author.unique()))

#new dataframe without problems 
nyt_single = nyt[nyt.problem == False]
nyt_single.to_csv('nyt_single_authors.csv', index=False)

how many problem entries 2416
0.04000927367270559
how many unique problems 189


In [90]:
# editors
#has_edited = nyt_review.author.str.contains('edited', regex = True, case = False)
nyt_review['edited'] = nyt_review.author.str.contains('edited', regex = True, case = False)
nyt_review.edited.sum()

# new dataframe with just editors
nyt_edited = nyt_review[nyt_review.edited == True]
print(nyt_edited.author.unique())

['Gideon Wyck, edited by Alexander Laing' 'edited by Somerset Maugham'
 'edited by Armin L. Robinson' 'edited by Nicholas Meyer'
 'edited by William J. Bennett. Illustrated by Michael Hague'
 'Margaret Mitchell. Edited by Debra Freer'
 'edited by John Updike and Katrina Kenison' 'edited by Joyce Reardon'
 'Hannah Crafts. Edited by Henry Louis Gates Jr'
 'J. R. R. Tolkien. Edited by Christopher Tolkien. Illustrated by Alan Lee'
 'J. R. R. Tolkien. Edited by Christopher Tolkien'
 'edited by Charlaine Harris and Toni L. P. Kelner'
 'edited by George R. R. Martin and Gardner Dozois'
 'Lucia Berlin, edited by Stephen Emerson']


/var/folders/p7/4d0x8hyj4mb4xypm89k57lrr0000gn/T/ipykernel_69500/4121782298.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyt_review['edited'] = nyt_review.author.str.contains('edited', regex = True, case = False)


In [ ]:
# if there's an author with editor, remove editor, else treat editor as author 
nyt_edited['author_alt'] = 

if name[:10] == 'edited by ':
    remove 'edited by ' and transfer rest of string to the 'author_alt' column
if name[:10] != 'edited by '"
    

In [12]:
# handle complicated authors

In [14]:
# authors + illustrators

In [ ]:
nyt_multi['three'] = nyt_multi.author.str.contains(',| with ', regex = True)
print(nyt_multi.three.sum())

nyt_two = nyt_multi[nyt_multi.three != True]
nyt_three_more = nyt_multi[nyt_multi.three == True]

In [ ]:
# sharing last name (i.e., Janet and Allan Ahlberg

# same author different spelling (i.e., James Norman Hall or James N. Hall)

# extraneous words (i.e. 'Edited by ', 'Illustrated by '
